# Sale Pipeline

In [ ]:
%pip install crewai crewai-tools

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew, LLM

## LLM Used (for development process in local)

In [3]:
llm = LLM(model="ollama/qwen2.5:3b", base_url="http://localhost:11434")

In [2]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_pipeline_lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_pipeline_lead_qualification_tasks.yaml',
    'email_agents': 'config/lead_pipeline_email_engagement_agents.yaml',
    'email_tasks': 'config/lead_pipeline_email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

## Create Pydantic model for structure output

In [3]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

## Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

c:\Users\Phat Dao\Documents\repo\personal_repository\learning\crewai\.venv\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\Phat Dao\Documents\repo\personal_repository\learning\crewai\.venv\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\Phat Dao\Documents\repo\personal_repository\learning\crewai\.venv\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: Pydan

## Lead Qualification Crew

In [5]:
# Creating Agents
# command llm=llm after locally development process to use OpenAI
lead_data_agent = Agent(
    config=lead_agents_config["lead_data_agent"]
    , tools=[SerperDevTool(), ScrapeWebsiteTool()]
    # , llm=llm
)

cultural_fit_agent = Agent(
    config=lead_agents_config["cultural_fit_agent"]
    , tools=[SerperDevTool(), ScrapeWebsiteTool()]
    # , llm=llm
)

scoring_validation_agent = Agent(
    config=lead_agents_config["scoring_validation_agent"]
    , tools=[SerperDevTool(), ScrapeWebsiteTool()]
    # , llm=llm
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

## Email Engagement Crew

In [6]:
# Creating Agents
# command llm=llm after locally development process to use OpenAI
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist']
  # , llm=llm
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist']
  # , llm=llm
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting']
  , agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization']
  , agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

## Complete Sale Flow (Lead Scoring => Email Engagement)

In [7]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # lead sample, can use any tool or anything to retrieve lead info here
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        print(f"==== \n DEBUG: \n{scores} \n ====")
        return scores
    
    @listen(score_leads)
    def store_leads_score(self, scores):
        # If there is a need to store score in database
        return scores
    
    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score["lead_score"].score > 70]
    
    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails
    
    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

In [8]:
flow = SalesPipeline()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: SalesPipeline                                                                                            │
│  ID: ce14dbe1-c084-4c16-92cf-59dce452c231                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Plotting the flow

In [9]:
flow.plot(filename="output/sale_pipeline.html")

Plot saved as output/sale_pipeline.html.html


In [10]:
from IPython.display import IFrame

IFrame(src='./output/sale_pipeline.html', width='150%', height=600)

## Kickoff the flow

In [11]:
import nest_asyncio
nest_asyncio.apply()

In [13]:
emails = flow.kickoff()

🌊 Flow: SalesPipeline
    ID: ce14dbe1-c084-4c16-92cf-59dce452c231
└── 🧠 Starting Flow...

 Flow started with ID: ce14dbe1-c084-4c16-92cf-59dce452c231


🌊 Flow: SalesPipeline
    ID: ce14dbe1-c084-4c16-92cf-59dce452c231
├── 🧠 Starting Flow...
└── 🔄 Running: fetch_leads

🌊 Flow: SalesPipeline
    ID: ce14dbe1-c084-4c16-92cf-59dce452c231
├── Flow Method Step
└── ✅ Completed: fetch_leads

🌊 Flow: SalesPipeline
    ID: ce14dbe1-c084-4c16-92cf-59dce452c231
├── Flow Method Step
├── ✅ Completed: fetch_leads
└── 🔄 Running: score_leads

...

🌊 Flow: SalesPipeline
    ID: ce14dbe1-c084-4c16-92cf-59dce452c231
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
└── ✅ Completed: write_email

🌊 Flow: SalesPipeline
    ID: ce14dbe1-c084-4c16-92cf-59dce452c231
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── 🔄 Running: send_email

🌊 Flow: SalesPipeline
    ID: ce14dbe1-c084-4c16-92cf-59dce452c231
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── ✅ Completed: send_email

✅ Flow Finished: SalesPipeline
├── Flow Method Step
├── ✅ Completed: fetch_leads
├── ✅ Completed: score_leads
├── ✅ Completed: store_leads_score
├── ✅ Completed: filter_leads
├── ✅ Completed: write_email
└── ✅ Completed: send_email

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: SalesPipeline                                                                                            │
│  ID: ce14dbe1-c084-4c16-92cf-59dce452c231                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Usage Metrics and Costs

In [ ]:
emails[0].token_usage

UsageMetrics(total_tokens=1233, prompt_tokens=1012, cached_prompt_tokens=0, completion_tokens=221, successful_requests=2)

In [27]:
import pandas as pd

token_usage = emails[0].token_usage

# $0.150 is cost for OpenAI model, this code just an example
costs = costs = 0.150 * (token_usage.prompt_tokens + token_usage.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([token_usage.dict()])
df_usage_metrics

Total costs: $0.0002


C:\Users\Phat Dao\AppData\Local\Temp\ipykernel_10080\3011958667.py:10: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  df_usage_metrics = pd.DataFrame([token_usage.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,1233,1012,0,221,2
